# Actor Critic

In [1]:
# import argparse
# import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [16]:
class env():
    def __init__(self):
        self.angle = 0
        self.r = 0
        self.done = False

        self.ang_range = 45
        self.ev_array = np.ones((31,31,30))
        
    def reset(self):
        #ここでもrを計算するために線量分布を計算した方が良いかも？
        self.r = 0
        self.angle = 0
        self.done = False
#         cmd = "source /opt/geant4/re10.07.p03-mt/bin/geant4.sh; ./bin/Application_Main 100 -phi " + str(self.angle)
#         test = subprocess.check_output(cmd, shell = True, executable = "/bin/bash")

#         dose = pd.readcsv(filename)['dose']
#         dose = dose.reshape(31,31,30)
        state = torch.rand(1,1,30,30,30)
        return state

    def step(self,action):
        self.angle += 5 * (action - 1)
        
        if (np.abs(self.angle) > self.ang_range):
            self.done = True
        
        next_state = np.random.rand(30,30,30)

        # cmd = "source /opt/geant4/re10.07.p03-mt/bin/geant4.sh; ./bin/Application_Main 100 -phi " + str(self.angle)
        # test = subprocess.check_output(cmd, shell = True, executable = "/bin/bash")
        
#         dose = pd.readcsv(filename)['dose']
#         dose = dose.reshape(31,31,30)
        
        reword = self.ev_array * next_state
        reword = reword.sum() 
        # self.r = reword - self.r
        
        return self.dose, reword, self.done

In [3]:
class PolicyNet(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self, nch_g=64):
        super(PolicyNet, self).__init__()
        
        self.layers = nn.ModuleDict({
            'Layer1': nn.Sequential(
                nn.Conv3d(1, nch_g, 2),
                # nn.BatchNorm3d(nch_g),
                nn.ReLU()
            ),
            'Layer2': nn.Sequential(
                nn.Conv3d(nch_g, nch_g * 2, 2),
                # nn.BatchNorm3d(nch_g * 2),
                nn.ReLU()
            ),
            'Layer3': nn.Sequential(
                nn.Conv3d(nch_g * 2, nch_g * 4, 2),
                # nn.BatchNorm3d(nch_g * 4),
                nn.ReLU()
            ),
            'Layer4': nn.Sequential(
                nn.Flatten(),
                nn.Linear(5038848, 3),#引数の計算が不要になるといい。
                nn.Softmax(dim=1)
            )
        })

    def forward(self, x):
        for layer in self.layers.values():
            x = layer(x)
        return x

In [4]:
pi = PolicyNet()

sample_input = torch.randn((1, 1, 30, 30, 30))

sample_out = pi(sample_input)

print(sample_out)

probs = pi(sample_input)

# action = torch.multinomial(probs, 1).item()

# print(probs, action)

print(pi.layers)

tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
ModuleDict(
  (Layer1): Sequential(
    (0): Conv3d(1, 64, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
  )
  (Layer2): Sequential(
    (0): Conv3d(64, 128, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
  )
  (Layer3): Sequential(
    (0): Conv3d(128, 256, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
  )
  (Layer4): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=5038848, out_features=3, bias=True)
    (2): Softmax(dim=1)
  )
)


In [5]:
class ValueNet(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self, nch_g=64):
        super(ValueNet, self).__init__()
        
        self.layers = nn.ModuleDict({
            'LayerV1': nn.Sequential(
                nn.Conv3d(1, nch_g, 2),
                # nn.BatchNorm3d(nch_g),
                nn.ReLU()
            ),
            'LayerV2': nn.Sequential(
                nn.Conv3d(nch_g, nch_g * 2, 2),
                # nn.BatchNorm3d(nch_g * 2),
                nn.ReLU()
            ),
            'LayerV3': nn.Sequential(
                nn.Conv3d(nch_g * 2, nch_g * 4, 2),
                # nn.BatchNorm3d(nch_g * 4),
                nn.ReLU()
            ),
            'LayerV4': nn.Sequential(
                nn.Flatten(),
                nn.Linear(5038848, 1)
            )
        })

    def forward(self, x):
        for layer in self.layers.values():
            x = layer(x)
        return x

In [6]:
v = ValueNet()

sample_input = torch.randn((1, 1, 30, 30, 30))

sample_out = v(sample_input)

print(sample_out)
print(v.layers)

tensor([[-0.0169]], grad_fn=<AddmmBackward>)
ModuleDict(
  (LayerV1): Sequential(
    (0): Conv3d(1, 64, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
  )
  (LayerV2): Sequential(
    (0): Conv3d(64, 128, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
  )
  (LayerV3): Sequential(
    (0): Conv3d(128, 256, kernel_size=(2, 2, 2), stride=(1, 1, 1))
    (1): ReLU()
  )
  (LayerV4): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=5038848, out_features=1, bias=True)
  )
)


In [14]:
class Agent:
    def __init__(self):
        self.gamma = 0.98
        self.lr_pi = 0.0002
        self.lr_v = 0.0005
        self.action_size = 3

        self.pi = PolicyNet()
        self.v = ValueNet()
        self.optimizer_pi = optim.Adam(self.pi.parameters(), lr=self.lr_pi)
        self.optimizer_v = optim.Adam(self.v.parameters(), lr=self.lr_v)
        
    def get_action(self, state):
        # state = torch.from_numpy(state).float()
        probs = self.pi.forward(state)
        m = Categorical(probs)
        action = m.sample()
        # action = torch.multinomial(probs, 1).item()
        return action, probs

    def update(self, state, action_prob, reward, next_state, done):
        # state = torch.from_numpy(state).float()
        next_state = torch.randn(1, 1, 30, 30, 30)

        # ========== (1) Update V network ===========
        target = reward + self.gamma * self.v(next_state) * (1 - done)
        v = self.v(state)
        loss_v = F.mse_loss(v, target.detach())
        
        # ========== (2) Update pi network ===========
        # with torch.no_grad():
        delta = target - v
        delta.detach()
        # loss_pi = -Categorical(probs).log_prob(action) * delta.detach()
        
        # t = reward + self.gamma * self.pi(next_state) * (1 - done)
        print(action_prob)
        loss_pi = -torch.log(action_prob[:,action]) * delta.detach()
        print(loss_pi)
        print(probs)
        loss = loss_pi + loss_v
        self.optimizer_v.zero_grad()
        self.optimizer_pi.zero_grad()
        # loss_v.backward()
        # loss_pi.backward(retain_graph=True)
        
        loss.backward(retain_graph=True)
        
        self.optimizer_v.step()
        self.optimizer_pi.step()

**渡瀬先生が使用しているupdateメソッド**

~~~
    def update(self, state, action_prob, reward, next_state, done):
        #state = state[np.newaxis, :]  # add batch axis ???
        #next_state = next_state[np.newaxis, :]???

        # ========== (1) Update V network ===========
        _,_,_,v1 = self.ac_net(state)
        _,_,_,v2 = self.ac_net(next_state)
        target = reward + opt.gamma * v2 * (1 - done)
        target.detach()
        loss_v = mse_loss(v1, target)
        
        # ========== (2) Update pi network ===========
        diff = target - v1
        diff.detach()
        loss_pi =torch.mean(-torch.log(action_prob) * diff)
        loss = loss_v + loss_pi
        self.optimizer_ac.zero_grad()
        loss.backward()
        self.optimizer_ac.step()

~~~

In [17]:
torch.autograd.set_detect_anomaly(True)

state = torch.randn(1, 1, 30, 30, 30)
reward = 500
next_state = torch.randn(1, 1, 30, 30, 30)
done = False

agent = Agent()

# get_actionメソッドのテスト
action, chosen_prob = agent.get_action(state)
print("Selected Action:", action)
print("Probability of the Selected Action:", probs)

# updateメソッドのテスト
agent.update(state, probs, reward, next_state, done)
print("Update Successful")

#==================ここから2回目=======================================
torch.manual_seed(0)
state = torch.randn(1, 1, 30, 30, 30)
next_state = torch.randn(1, 1, 30, 30, 30)

# get_actionメソッドのテスト
action, chosen_prob = agent.get_action(state)
print("Selected Action:", action)
print("Probability of the Selected Action:", probs)

# updateメソッドのテスト
agent.update(state, probs, reward, next_state, done)
print("Update Successful")

#==================ここから2回目=======================================
torch.manual_seed(2010)
state = torch.randn(1, 1, 30, 30, 30)
print("state : ",state[0,0,2,2,2])
next_state = torch.randn(1, 1, 30, 30, 30)

# get_actionメソッドのテスト
action, chosen_prob = agent.get_action(state)
print("Selected Action:", action)
print("Probability of the Selected Action:", probs)

# updateメソッドのテスト
agent.update(state, probs, reward, next_state, done)
print("Update Successful")

#==================ここから2回目=======================================
torch.manual_seed(99981)
state = torch.randn(1, 1, 30, 30, 30)
print("state : ",state[0,0,2,2,2])
next_state = torch.randn(1, 1, 30, 30, 30)

# get_actionメソッドのテスト
action, chosen_prob = agent.get_action(state)
print("Selected Action:", action)
print("Probability of the Selected Action:", probs)

# updateメソッドのテスト
agent.update(state, probs, reward, next_state, done)
print("Update Successful")

Selected Action: tensor([2])
Probability of the Selected Action: tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
tensor([[529.7750]], grad_fn=<MulBackward0>)
tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
Update Successful
Selected Action: tensor([2])
Probability of the Selected Action: tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
tensor([[527.7347]], grad_fn=<MulBackward0>)
tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
Update Successful
state :  tensor(2.1180)
Selected Action: tensor([1])
Probability of the Selected Action: tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
tensor([[566.7744]], grad_fn=<MulBackward0>)
tensor([[0.3334, 0.3200, 0.3466]], grad_fn=<SoftmaxBackward>)
Update Successful
state :  tensor(0.2289)
Selected A

In [18]:
print(torch.__version__)

1.9.0+cu111
